In [54]:
import requests
import pandas as pd

# Insert your API key
API_KEY = "3615cfc7361a4adb726311266cf81466"

In [55]:
# Top 10 world cities
CITIES = [
    {"city": "Tokyo", "lat": 35.6895, "lon": 139.6917, "population": 13929286},
    {"city": "Delhi", "lat": 28.7041, "lon": 77.1025, "population": 27882721},
    {"city": "Shanghai", "lat": 31.2304, "lon": 121.4737, "population": 24256800},
    {"city": "São Paulo", "lat": -23.5505, "lon": -46.6333, "population": 21571281},
    {"city": "Mumbai", "lat": 19.0760, "lon": 72.8777, "population": 21558265},
    {"city": "Mexico City", "lat": 19.4326, "lon": -99.1332, "population": 21782378},
    {"city": "Beijing", "lat": 39.9042, "lon": 116.4074, "population": 21516000},
    {"city": "Osaka", "lat": 34.6937, "lon": 135.5023, "population": 19222665},
    {"city": "New York", "lat": 40.7128, "lon": -74.0060, "population": 18804000},
    {"city": "Cairo", "lat": 30.0444, "lon": 31.2357, "population": 20095952}
  ]

In [56]:

# Create an empty list to store air pollution data for each city
air_pollution_data_list = []

# Loop through the cities and make API calls for historical air pollution data
for city in CITIES:
    lat = city["lat"]
    lon = city["lon"]
    city_name = city["city"]

    # Set the start and end dates for historical data
    start = 1641016800  # Start date 01-01-2022
    end = 1672552799  #  End date 31-12-2022

    # Make API call
    url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={API_KEY}"
    response = requests.get(url)

    # Check if API call was successful
    if response.status_code == 200:
        # Extract air pollution data from response
        air_pollution_data = response.json()

        # Append the air pollution data for this city to the final list
        air_pollution_data_list.append(air_pollution_data)

In [57]:
air_pollution_data_list

[{'coord': {'lon': 139.6917, 'lat': 35.6895},
  'list': [{'main': {'aqi': 2},
    'components': {'co': 270.37,
     'no': 1.98,
     'no2': 12.68,
     'o3': 87.26,
     'so2': 12.76,
     'pm2_5': 0.62,
     'pm10': 1.38,
     'nh3': 0.57},
    'dt': 1641016800},
   {'main': {'aqi': 1},
    'components': {'co': 280.38,
     'no': 1.31,
     'no2': 20.05,
     'o3': 78.68,
     'so2': 16.93,
     'pm2_5': 0.78,
     'pm10': 1.79,
     'nh3': 0.77},
    'dt': 1641020400},
   {'main': {'aqi': 1},
    'components': {'co': 297.07,
     'no': 0.12,
     'no2': 27.42,
     'o3': 69.38,
     'so2': 20.03,
     'pm2_5': 0.94,
     'pm10': 2.09,
     'nh3': 1.01},
    'dt': 1641024000},
   {'main': {'aqi': 1},
    'components': {'co': 310.42,
     'no': 0.01,
     'no2': 30.85,
     'o3': 62.94,
     'so2': 21.93,
     'pm2_5': 1.06,
     'pm10': 2.24,
     'nh3': 1.19},
    'dt': 1641027600},
   {'main': {'aqi': 1},
    'components': {'co': 320.43,
     'no': 0.02,
     'no2': 36.67,
     'o3'

In [105]:
air_pollution_df = pd.json_normalize(air_pollution_data_list,record_path="list", meta=['coord'])
air_pollution_df

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,coord
0,1641016800,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57,"{'lon': 139.6917, 'lat': 35.6895}"
1,1641020400,1,280.38,1.31,20.05,78.68,16.93,0.78,1.79,0.77,"{'lon': 139.6917, 'lat': 35.6895}"
2,1641024000,1,297.07,0.12,27.42,69.38,20.03,0.94,2.09,1.01,"{'lon': 139.6917, 'lat': 35.6895}"
3,1641027600,1,310.42,0.01,30.85,62.94,21.93,1.06,2.24,1.19,"{'lon': 139.6917, 'lat': 35.6895}"
4,1641031200,1,320.43,0.02,36.67,55.79,26.23,1.17,2.48,1.38,"{'lon': 139.6917, 'lat': 35.6895}"
...,...,...,...,...,...,...,...,...,...,...,...
85915,1672534800,2,317.10,0.02,23.99,28.61,12.76,23.35,48.37,2.50,"{'lon': 31.2357, 'lat': 30.0444}"
85916,1672538400,2,313.76,0.02,22.28,27.18,12.28,23.23,43.94,2.47,"{'lon': 31.2357, 'lat': 30.0444}"
85917,1672542000,2,310.42,0.03,21.25,28.25,12.16,22.07,39.83,2.41,"{'lon': 31.2357, 'lat': 30.0444}"
85918,1672545600,2,313.76,0.04,22.62,28.25,13.83,20.83,36.78,2.53,"{'lon': 31.2357, 'lat': 30.0444}"


In [106]:
air_pollution_df.columns

Index(['dt', 'main.aqi', 'components.co', 'components.no', 'components.no2',
       'components.o3', 'components.so2', 'components.pm2_5',
       'components.pm10', 'components.nh3', 'coord'],
      dtype='object')

In [107]:
air_pollution_df

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,coord
0,1641016800,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57,"{'lon': 139.6917, 'lat': 35.6895}"
1,1641020400,1,280.38,1.31,20.05,78.68,16.93,0.78,1.79,0.77,"{'lon': 139.6917, 'lat': 35.6895}"
2,1641024000,1,297.07,0.12,27.42,69.38,20.03,0.94,2.09,1.01,"{'lon': 139.6917, 'lat': 35.6895}"
3,1641027600,1,310.42,0.01,30.85,62.94,21.93,1.06,2.24,1.19,"{'lon': 139.6917, 'lat': 35.6895}"
4,1641031200,1,320.43,0.02,36.67,55.79,26.23,1.17,2.48,1.38,"{'lon': 139.6917, 'lat': 35.6895}"
...,...,...,...,...,...,...,...,...,...,...,...
85915,1672534800,2,317.10,0.02,23.99,28.61,12.76,23.35,48.37,2.50,"{'lon': 31.2357, 'lat': 30.0444}"
85916,1672538400,2,313.76,0.02,22.28,27.18,12.28,23.23,43.94,2.47,"{'lon': 31.2357, 'lat': 30.0444}"
85917,1672542000,2,310.42,0.03,21.25,28.25,12.16,22.07,39.83,2.41,"{'lon': 31.2357, 'lat': 30.0444}"
85918,1672545600,2,313.76,0.04,22.62,28.25,13.83,20.83,36.78,2.53,"{'lon': 31.2357, 'lat': 30.0444}"


In [108]:
# Define a function to split 'coord' column into 'longitude' and 'latitude'
def split_coord(coord):
    return pd.Series([coord['lon'],coord['lat']])

# Apply the function to 'coord' column to create 'longitude' and 'latitude' columns
air_pollution_df[['longitude', 'latitude']] = air_pollution_df['coord'].apply(split_coord)

# Drop the original 'coord' column
air_pollution_df.drop('coord',axis=1,inplace=True)

In [109]:
cities_df = pd.DataFrame(CITIES)
cities_df.columns

Index(['city', 'lat', 'lon', 'population'], dtype='object')

In [110]:
new_columns_name = ['city', 'latitude', 'longitude', 'population']
cities_df.columns= new_columns_name
cities_df

,city,latitude,longitude,population
0,Tokyo,35.6895,139.6917,13929286
1,Delhi,28.7041,77.1025,27882721
2,Shanghai,31.2304,121.4737,24256800
3,São Paulo,-23.5505,-46.6333,21571281
4,Mumbai,19.0760,72.8777,21558265
5,Mexico City,19.4326,-99.1332,21782378
6,Beijing,39.9042,116.4074,21516000
7,Osaka,34.6937,135.5023,19222665
8,New York,40.7128,-74.0060,18804000
9,Cairo,30.0444,31.2357,20095952


In [113]:
air_pollution_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85920 entries, 0 to 85919
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   dt                85920 non-null  int64  
 1   main.aqi          85920 non-null  int64  
 2   components.co     85920 non-null  float64
 3   components.no     85920 non-null  float64
 4   components.no2    85920 non-null  float64
 5   components.o3     85920 non-null  float64
 6   components.so2    85920 non-null  float64
 7   components.pm2_5  85920 non-null  float64
 8   components.pm10   85920 non-null  float64
 9   components.nh3    85920 non-null  float64
 10  longitude         85920 non-null  float64
 11  latitude          85920 non-null  float64
dtypes: float64(10), int64(2)
memory usage: 7.9 MB


In [114]:
air_pollution_df = pd.merge(air_pollution_df,cities_df, left_on='latitude',right_on='latitude')
air_pollution_df

,dt,main.aqi,components.co,components.no,components.no2,components.o3,components.so2,components.pm2_5,components.pm10,components.nh3,longitude_x,latitude,city,longitude_y,population
0,1641016800,2,270.37,1.98,12.68,87.26,12.76,0.62,1.38,0.57,139.6917,35.6895,Tokyo,139.6917,13929286
1,1641020400,1,280.38,1.31,20.05,78.68,16.93,0.78,1.79,0.77,139.6917,35.6895,Tokyo,139.6917,13929286
2,1641024000,1,297.07,0.12,27.42,69.38,20.03,0.94,2.09,1.01,139.6917,35.6895,Tokyo,139.6917,13929286
3,1641027600,1,310.42,0.01,30.85,62.94,21.93,1.06,2.24,1.19,139.6917,35.6895,Tokyo,139.6917,13929286
4,1641031200,1,320.43,0.02,36.67,55.79,26.23,1.17,2.48,1.38,139.6917,35.6895,Tokyo,139.6917,13929286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85915,1672534800,2,317.10,0.02,23.99,28.61,12.76,23.35,48.37,2.50,31.2357,30.0444,Cairo,31.2357,20095952
85916,1672538400,2,313.76,0.02,22.28,27.18,12.28,23.23,43.94,2.47,31.2357,30.0444,Cairo,31.2357,20095952
85917,1672542000,2,310.42,0.03,21.25,28.25,12.16,22.07,39.83,2.41,31.2357,30.0444,Cairo,31.2357,20095952
85918,1672545600,2,313.76,0.04,22.62,28.25,13.83,20.83,36.78,2.53,31.2357,30.0444,Cairo,31.2357,20095952
